# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import 

sns.set_style('dark')
%matplotlib inline

#create dataframe from 'homepage_actions.csv'
df = pd.read_csv('homepage_actions.csv')
df.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [34]:
print('rows in data frame:', len(df))

rows in data frame: 8188


In [35]:
#explorle id 
df['id'].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [39]:
# Some users have multiple entries.  Assuming view and click.. 
df[df['id']==363314]

,timestamp,id,group,action
3751,2016-11-17 15:51:56.634476,363314,control,view
3752,2016-11-17 15:52:30.776850,363314,control,click


In [45]:
#Check to see if anyone viewed who didn't click..
# viewers = clickers + non_clickers v=c+nc
v = sum(df['action']=='view')
c = sum(df['action']=='click')
v,c



(6328, 1860)

In [48]:
# Need to use sets of unique ids to actually know if there are any users who viewed without clicking
set_v = set(df[df['action']=='view']['id'].unique())
set_c = set(df[df['action']=='click']['id'].unique())
v_count = len(set_v)
c_count = len(set_c)
print('viewers: {} \t clickers: {}'.format(v_count, c_count))

viewers: 6328 	 clickers: 1860


In [51]:
# viewers - clickers gives non clickers
#clickers - viewers should equal zero if there unless there is a clicker who didn't view
set_nc = set_v - set_c
set_vwc = set_c - set_v
print('Non Clickers: {} \t Viewers without Clicking: {}'.format(len(set_nc),len(set_vwc)))
if len(set_vwc)==0:
    print('No anomoly with click data')

Non Clickers: 4468 	 Viewers without Clicking: 0
No anomoly with click data


In [67]:
#get value counts for groups
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [68]:
##Is there any overlap between control and experiment group -
#An overlap would be an id existing in both groups
overlap_df = pd.merge(df_c,df_e, how='inner', on='id')
if len(overlap_df)==0:
    print('There are {} rows in the overlap. NO OVERLAP FOUND'.format(len(overlap_df)))
else:
    print('There are {} rows in the overlap. OVERLAP FOUND'.format(len(overlap_df)))

There are 0 rows in the overlap. NO OVERLAP FOUND


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [81]:
#Null: There is no difference in clicks for the control and experimental groups at an alpha of 0.05
#Alt: There is a difference in clicks for the control and expeirmental groups at an alpha of 0.05

In [64]:
#Need to add view/ click columns and set initial values = 0
df['view'] = 0
df['click'] = 0
df.head()
df.loc[df['action']== 'view', "view"] = 1
df.loc[df['action']== 'click', "click"] = 1
df.head(10)

,timestamp,id,group,action,view,click
0,2016-09-24 17:42:27.839496,804196,experiment,view,1,0
1,2016-09-24 19:19:03.542569,434745,experiment,view,1,0
2,2016-09-24 19:36:00.944135,507599,experiment,view,1,0
3,2016-09-24 19:59:02.646620,671993,control,view,1,0
4,2016-09-24 20:26:14.466886,536734,experiment,view,1,0
5,2016-09-24 20:32:25.712659,681598,experiment,view,1,0
6,2016-09-24 20:39:03.248853,522116,experiment,view,1,0
7,2016-09-24 20:57:20.336757,349125,experiment,view,1,0
8,2016-09-24 20:58:01.948663,349125,experiment,click,0,1
9,2016-09-24 21:00:12.278374,560027,control,view,1,0


In [76]:
#create 4 groups for chi square test
# control click, control nonclick, experiment click, experiment nonclick
#use df_c and df_e 
df_c = df[df['group']=='control']
df_e = df[df['group']=='experiment']

#clickers
c_click = sum(df_c['click'])  #control group clickers
e_click = sum(df_e['click']) #experiment clickers

#nonclickers
c_nonclick = len(df_c) - c_click
e_nonclick = len(df_e) - e_click
c_click, e_click, c_nonclick, e_nonclick
print('Control Group {} clicks \t {} non_clicks \nExperiment Group {} clicks \t {} non_clicks'
      .format(c_click,c_nonclick,e_click,e_nonclick))

Control Group 932 clicks 	 3332 non_clicks 
Experiment Group 928 clicks 	 2996 non_clicks


In [80]:
#chi square test for independence
obs = np.array([[c_click, c_nonclick],
               [e_click, e_nonclick]])
result = stats.contingency.chi2_contingency(obs)
chi, p, dof, exp = result
print('Chi stat: {} \np: {} \nDegree of Freedom: {} \nExpected: {}'.format(chi,p,dof,exp))



Chi stat: 3.636160051233291 
p: 0.056537191086915774 
Degree of Freedom: 1 
Expected: [[ 968.61748901 3295.38251099]
 [ 891.38251099 3032.61748901]]


In [ ]:
#p value of 0.056 is higher than 0.05 - Fail to reject Null hypothesis

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [93]:
#expected clicks for experiment group
control_rate = df_c.click.mean()
expected_exp_rate = exp[1][0] / (exp[1][0]+exp[1][1])
print('Control Click Rate: {}\nExperimental Click Rate: {}'.format(control_rate,expected_exp_rate))
print('Expected clicks from Chi square test:', exp[1][0])


Control Click Rate: 0.21857410881801126
Experimental Click Rate: 0.227161700048852
Expected clicks from Chi square test: 891.3825109916952


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [95]:
#std = np.sqrt(var)
n=len(df_e)
p=control_rate
var = n * p * 1-p
std=np.sqrt(var)
print('STD:', std)

STD: 29.28252429168391


In [97]:
#get a z score
z = e_click - exp[1][0]
z

36.6174890083048

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [98]:
#p value from normal distribution based on zscore
p_value = stats.norm.sf(z)
p_value

7.534992124934037e-294

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: My results do not double check.  I would have rejected the Null based on the chi square test.  The pvalue in my double check is much lower.  I am guessing i messed up at least one of the tests.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.